In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import cv2
import numpy as np
from utils import *
from frame import extract,match,Frame


In [3]:
np.set_printoptions(suppress=True)

In [4]:
W,H=640,480
K=np.array([[1,0,W//2],[0,1,H//2],[0,0,1]])

In [5]:
class Map(object):
    def __init__(self):
        self.frames=[]
        self.points=[]
    def display(self):
        for f in self.frames:
            print(f.id)
            print(f.pose)

In [6]:
frames=[]
mapp=Map()

In [7]:

def process_img(img,depth):

    frame=Frame(mapp,img,depth,K)
    frames.append(frame)


    if len(frames)<=1:
        return
    f_c=frames[-1] #current frame
    f_p=frames[-2] # previous frame
    
    
    idx1,idx2,pose=match(f_p,f_c)
    
    
    # idx1-> id of the keypoint in previous frame
    # idx2-> id of the keypoint in current frame
    # pose relative transformation of current frame  
    # with respect to previous frame 

    assert len(idx1)>0
    assert len(idx2)>0
    
    if pose is None:
        return
    


    f_c.pose=np.dot(pose,f_p.pose)
    # print(f_c.pose)

    pts4d=add_ones(f_c.kps)
    pts4d=np.dot(f_p.pose,pts4d.T)


    for kp1,kp2 in zip(f_p.kps[idx1],f_c.kps[idx2]):
        u_p,v_p,_=denormalize(kp1,f_p.K)
        u_c,v_c,_=denormalize(kp2,f_c.K)
        # u_p,v_p,_=kp1

        cv2.circle(img,(u_p,v_p),color=(0,255,0),radius=3)
        cv2.line(img,(u_p,v_p),(u_c,v_c),color=(255,0,0))
    disp(img,"RGB")
    mapp.display()




In [8]:
if __name__ == "__main__":
    
    depth_paths='../dataset/rgbd_dataset_freiburg1_xyz/depth.txt'
    dlist=data(depth_paths)

    rgb_paths='../dataset/rgbd_dataset_freiburg1_xyz/rgb.txt'
    ilist=data(rgb_paths)

    dataset_path='../dataset/rgbd_dataset_freiburg1_xyz/'
    

    for i in range(len(dlist)):

        frame=cv2.imread(dataset_path+ilist[i])
        depth=cv2.imread(dataset_path+dlist[i],0)
        # print(frame.shape,depth.shape)
        W=frame.shape[1]
        H=frame.shape[0]
        # print(W,H)
        process_img(frame,depth)
        # print(frame)

        if frame is None:
            print("End of frame")
            break

        
        disp(depth,"Depth")
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()


        


0
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
1
[[ 1.00545866 -0.0012372   0.00161193  4.35850749]
 [ 0.00123446  1.0054585   0.00170718 -5.1646544 ]
 [-0.00161403 -0.0017052   1.00545797 -0.47187143]
 [ 0.          0.          0.          1.        ]]
0
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
1
[[ 1.00545866 -0.0012372   0.00161193  4.35850749]
 [ 0.00123446  1.0054585   0.00170718 -5.1646544 ]
 [-0.00161403 -0.0017052   1.00545797 -0.47187143]
 [ 0.          0.          0.          1.        ]]
2
[[  1.0149041   -0.0032741    0.00200193   8.12105457]
 [  0.00327401   1.01490607   0.00004855 -11.82629845]
 [ -0.00200208  -0.00004209   1.01490938  -0.90304643]
 [  0.           0.           0.           1.        ]]
0
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
1
[[ 1.00545866 -0.0012372   0.00161193  4.35850749]
 [ 0.00123446  1.0054585   0.00170718 -5.1646544 ]
 [-0.00161403 -0.0017052   1.00545797 -0.47187143]
 [ 0.          0.  